In [1]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int):
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

In [2]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [3]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [4]:
show_lane(scenario)

KeyboardInterrupt: 

In [6]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-dur[0]), :, : ] = buffer
        print('fps:', (i - dur[0])/(time.time()-t1))
    return data

In [7]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10
draw_params = create_draw_params()

origin_data = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 12.763697004369895
fps: 17.172000286588673
fps: 20.32494948205987
fps: 23.095815356998312
fps: 25.690348993469463
fps: 27.6243951701427
fps: 28.392044460045817
fps: 28.750977446959098
fps: 29.587451678742408
fps: 30.06294555374455
fps: 31.015867808596745
fps: 31.858961772396597
fps: 32.60741241302599
fps: 33.24722761682058
fps: 33.8381854882301
fps: 34.326117446310505
fps: 34.468121063291875
fps: 34.33223254295311
fps: 34.703898168167974
fps: 34.58301383055387
fps: 34.49538731947571
fps: 34.550801738573014
fps: 34.15941605095036
fps: 33.328796050736514
fps: 32.448393464497656
fps: 31.54411123234226
fps: 30.89547159526641
fps: 29.91204257557253
fps: 29.235083499235802
fps: 28.773029675095504
fps: 28.412564394762978
fps: 28.06661574078237
fps: 27.70459223746376
fps: 27.21215050589262
fps: 26.777003980052314
fps: 26.491958456411023
fps: 26.19680565890206
fps: 25.92849990751648
fps: 25.753292254735168
fps: 25.541925014550504
fps: 25.338312398626986
fps: 24.96082431572698
fps:

In [8]:
def plt_data(observe):
    fig = plt.figure()
    plt.gray()
    if len(observe.shape) <=2:
        plt.imshow(observe)
    elif len(observe.shape) > 3:
        for r in observe:
            myobj = plt.imshow(r[0, :, :])
            for rr in r:
                myobj.set_data(rr)
                plt.draw()
                plt.pause(0.01)
    else:
        myobj = plt.imshow(observe[0, :, :])
        for row in observe:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [12]:
from commonroad.scenario.scenario import Scenario

ego_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
ego_scenario.lanelet_network = scenario.lanelet_network

In [62]:
def theTrueData(leng, start_egoid, dt):
    #  初始化fig
    set_non_blocking()  # ensures interactive plotting is activated
    size = [400, 30]
    pos = [350, 135]
    inch_in_cm = 10
    fig = plt.figure(figsize=(size[0] / inch_in_cm, size[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')

    #     draw scenario
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    draw_params = create_draw_params()
    draw_object(scenario.lanelet_network, handles=handles, draw_params=draw_params)
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    #   初始化data
    start_goal = np.zeros((leng, 8), dtype=np.float64)
    observed = np.zeros(( (leng, dt,) + fig.canvas.get_width_height()[::-1]), dtype=np.bool)
    data = np.zeros((leng,  dt, 4), dtype=np.float64)

    #   从哪辆车开始呢
    ego_id = start_egoid
    index = 0
    while True:
        ego = None
        while(scenario.obstacle_by_id(ego_id) == None):
            ego_id = ego_id + 1
        ego = scenario.obstacle_by_id(ego_id)
        ego_id = ego_id + 1
        print("ego_id: ", ego_id)
    #   获得那辆车的traj，initial_time, terminated_time
        scenario.remove_obstacle(ego)
        traj = ego.prediction.trajectory
        initial_time = traj.initial_time_step
        terminated_time = traj.state_list[-1].time_step
    #  开始循环
        start_time = initial_time + 2* dt
        end_time = start_time + 2 * dt

        while end_time < terminated_time - 2* dt:
            t =int( (start_time - initial_time)/2)
            pos = traj.state_list[t].position + [100, 0]
    #   实际traj数据
            for i in range(0, dt):
                data[index, i, :] = [traj.state_list[t+i].position[0], traj.state_list[t+i].position[1], traj.state_list[t+i].orientation, traj.state_list[t+i].velocity]
    #   起点，终点
            start_goal[index, 0:4] = data[index, 0, :]
            start_goal[index, 4:8] = data[index, -1, :]

            plot_limits = [start_goal[index, 0] - size[0]/2, start_goal[index, 0] + size[0]/2, start_goal[index, 1]+-size[1]/2,start_goal[index, 1]+ size[1]/2]
            print("start_goal: ", start_goal[index, :])
            t1 = time.time()
            # loop where obstacle positions are updated
            for i in range(0, dt):
                # ...
                # change positions of obstacles
                # ...
                draw_params['time_begin'] = start_time + 2*i
                redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
                buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
                buffer = buffer[:, : , 0].astype(bool)
    #   observed data
                observed[index, i, :, : ] = buffer
    #                 print('fps:', i/(time.time()-t1), " time:", i)
            start_time = start_time + 2 * dt
            end_time = start_time + 2 * dt

            index = index + 1
            if(index == leng): break
        scenario.add_objects(ego)
        if(index == leng): break
    return start_goal, observed, data, ego_id + 1

In [63]:
leng = 10
dt = 25
start_ego = 0
start_goal_i, observed_i, data_i, last_ego =  theTrueData(leng, start_ego, dt)

ego_id:  120
start_goal:  [ 1.4498e+02  1.3982e+02 -0.0000e+00  2.9220e+01  2.7819e+02  1.3970e+02
 -7.3000e-03  2.9190e+01]
start_goal:  [ 2.8374e+02  1.3958e+02 -1.1000e-02  2.9190e+01  4.1898e+02  1.3662e+02
 -0.0000e+00  2.9270e+01]
start_goal:  [424.8  136.62  -0.    29.27 564.64 136.62  -0.    29.26]
start_goal:  [570.46 136.62  -0.    29.25 710.1  136.62  -0.    29.2 ]
start_goal:  [715.91 136.62  -0.    29.2  855.25 136.62  -0.    29.14]
start_goal:  [ 861.05  136.62   -0.     29.13 1000.08  136.62   -0.     29.07]
start_goal:  [1005.86  136.62   -0.     29.07 1144.56  136.62   -0.     29.  ]
start_goal:  [1150.33  136.62   -0.     29.   1288.69  136.62   -0.     28.93]
start_goal:  [1294.45  136.62   -0.     28.92 1432.45  136.62   -0.     28.85]
start_goal:  [1438.2   136.62   -0.     28.85 1575.85  136.62   -0.     28.78]
